In [75]:
using ReadableRegex
using Plots
using Makie, CairoMakie
currentpath = pwd()

"/home/davibarreira/MEGA/EMAp/PlutoLatexConverter.jl"

In [76]:
module Runner
end

Main.Runner

In [77]:
figureindex = 0

0

In [68]:
function extractnotebook(notebook, notebookname=nothing)
    s = read(notebook, String)
    cells = split(s, "# ╔═╡ ");
    # The first cell and the final 3 are not used
    codes, contents, outputtag, celltype = [],[],[],[]
    for cell in cells[2:end-3]
        push!(codes, cell[1:36])
        push!(contents, cell[38:end])
        push!(outputtag, endswith(rstrip(cell),";") ? "hideoutput" : "showoutput")
        push!(celltype, cell[38:42] == "md\"\"\"" ? "markdown" : "code")
    end
    
    # Get order and view type
    r = either("# ╠","# ╟")
    sortedcells = split(cells[end],Regex(r))
    sortedcodes = [cell[4:39] for cell in sortedcells[2:end-2]]
    order = [findfirst(isequal(scode),codes) for scode in  sortedcodes[1:end]]
    view  = [occursin("═",c) ? "showcode" : "hidecode" for c in sortedcells[2:end-2]]
    # Matching running order
    view  = view[order]

    # inferring the notebook name
    # base on the notebook file path.
    if notebookname === nothing
        notebookname = split(notebook,"/")[end]
        if endswith(notebookname,".jl")
            # removes the ".jl" in the end
            notebookname = notebookname[1:end-3]
        end
    end
    
    notebookdata = Dict(:codes => codes, :notebookname => notebookname,
                        :contents => contents, :outputtag=>outputtag,
                        :celltype => celltype,:order=> order,:view=>view)
    return notebookdata
end

function collectoutputs(notebookdata, notebookfolder="./")
    runpath = pwd()
    cd(notebookfolder)
    outputs = []
    for (i, content) ∈ enumerate(notebookdata[:contents])
        if notebookdata[:celltype][i] == "code"
            if startswith(lstrip(content),"begin") && endswith(rstrip(content),"end")
                ex = :($(Meta.parse(strip(content))))
            else
                ex = :($(Meta.parse("begin\n"*content*"\nend")))
            end
            
            s = string(ex.args[end])
            if contains(s, Regex(either("PlutoUI.LocalResource","LocalResource")))
                if findfirst(Regex(look_for(one_or_more(ANY),after="(\"",before="\")")),s) === nothing
                    Runner.eval(ex)
                    pathvariable = s[findfirst(Regex(look_for(one_or_more(ANY),after="(",before=")")),s)]
                    imagepath = Runner.eval(Meta.parse(pathvariable))
                else
                    imagepath = s[findfirst(Regex(look_for(one_or_more(ANY),after="(\"",before="\")")),s)]
                end
                push!(outputs,(:image,imagepath))
            else
                io = IOBuffer();
                Base.invokelatest(show,
                    IOContext(io, :limit => true),"text/plain",
                    dispatch_output(Runner.eval(ex), notebookdata[:notebookname]));
                celloutput = String(take!(io))
                if celloutput == "nothing"
                    push!(outputs,nothing)
                elseif startswith(celloutput, "Plot{Plots.")
                    push!(outputs,
                    (:plot,notebookdata[:notebookname]*"_"*"figure"*string(figureindex)*".png"))
                elseif startswith(celloutput, "FigureAxisPlot()")
                    push!(outputs,
                    (:plot,notebookdata[:notebookname]*"_"*"figure"*string(figureindex)*".png"))
                else
                    push!(outputs,(:text, celloutput))
                end
            end
        end
    end
    cd(runpath)
    return outputs
end

function dispatch_output(command_eval::Makie.FigureAxisPlot, notebookname)
    global figureindex+=1
    save(notebookname*"_"*"figure"*string(figureindex)*".png", command_eval)
    return command_eval
end

function dispatch_output(command_eval::Plots.Plot, notebookname)
    global figureindex+=1
    savefig(command_eval,notebookname*"_"*"figure"*string(figureindex)*".png")
    return command_eval
end

function dispatch_output(command_eval, notebookname)
   return command_eval 
end


dispatch_output (generic function with 3 methods)

In [71]:
example = "./example/examplepluto.jl"
nb = extractnotebook(example)

Dict{Symbol, Any} with 7 entries:
  :codes        => Any["7b4b693d-c955-4c9b-97e8-56d28cf74e18", "bdad72d3-5744-4…
  :outputtag    => Any["showoutput", "showoutput", "showoutput", "showoutput", …
  :celltype     => Any["code", "code", "code", "markdown", "code", "code", "cod…
  :view         => ["showcode", "showcode", "showcode", "showcode", "showcode",…
  :order        => [4, 1, 2, 5, 6, 7, 3, 8, 9, 10, 11, 12]
  :notebookname => "examplepluto"
  :contents     => Any["using PlutoUI\n\n", "\n\t begin\n\tusing Plots\n\ty(x) …

In [72]:
cd(currentpath)
outputs = collectoutputs(nb,"./example/");

┌ Warning: `LocalResource` **will not work** when you share the script/notebook with someone else, _unless they have those resources at exactly the same location on their file system_. 
│ 
│ ## Recommended alternatives (images)
│ 1. Go to [imgur.com](https://imgur.com) and drag&drop the image to the page. Right click on the image, and select "Copy image location". You can now use the image like so: `PlutoUI.Resource("https://i.imgur.com/SAzsMMA.jpg")`.
│ 2. If your notebook is part of a git repository, place the image in the repository and use a relative path: `PlutoUI.LocalResource("../images/cat.jpg")`.
└ @ PlutoUI /home/davibarreira/.julia/packages/PlutoUI/iYlHi/src/Resource.jl:80


In [73]:
outputs

11-element Vector{Any}:
 nothing
 (:plot, "examplepluto_figure19.png")
 nothing
 (:text, "3-element Vector{Int64}:\n 10\n 10\n 10")
 (:text, "10-element Vector{Float64}:\n 0.5804692302814431\n 0.8187680304211684\n 0.6094307577994669\n 0.7652475734611952\n 0.880076820645445\n 0.6968532202625504\n 0.8718613705930653\n 0.17962180619044954\n 0.7573695165430776\n 0.3845679297317386")
 (:text, "10-element Vector{Float64}:\n 1.5804692302814431\n 1.8187680304211684\n 1.6094307577994669\n 1.7652475734611952\n 1.880076820645445\n 1.6968532202625504\n 1.8718613705930653\n 1.1796218061904495\n 1.7573695165430776\n 1.3845679297317386")
 nothing
 (:plot, "examplepluto_figure20.png")
 (:image, "figure.svg")
 (:text, "\"plotexample.png\"")
 (:image, "plotexample.png")

In [82]:
save("./test/testplot.png",Makie.plot(rand(10)))

CairoScreen{Cairo.CairoSurfaceBase{UInt32}} with surface:
Cairo.CairoSurfaceBase{UInt32}(Ptr{Nothing} @0x0000000005198f40, 800.0, 600.0)
